<a href="https://colab.research.google.com/github/shivansh9604/Amazon_Hackathon/blob/main/amazon_hack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import requests

def download_images(image_links, save_dir):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    for idx, link in enumerate(image_links):
        img_data = requests.get(link).content
        with open(os.path.join(save_dir, f'image_{idx}.jpg'), 'wb') as handler:
            handler.write(img_data)

# Example usage:
train_image_links = [...]  # Replace with list of train image URLs
test_image_links = [...]   # Replace with list of test image URLs

# Download train and test images
download_images(train_image_links, 'train_images')
download_images(test_image_links, 'test_images')


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define directories
train_dir = 'train_images'
test_dir = 'test_images'

# Data augmentation and rescaling for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

# Only rescaling for test data (no augmentation)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load train data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

# Load test data
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode=None,  # No labels for the test set
    shuffle=False)


In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

# Load the pre-trained ResNet50 model without the top layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model's layers (you can unfreeze later if fine-tuning)
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers for classification
x = Flatten()(base_model.output)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()


In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=None)

# You can also create a validation set if needed by splitting the train set


In [ ]:
# Predict on test data
test_generator.reset()  # Reset the generator if it has been used
predictions = model.predict(test_generator, steps=test_generator.samples // test_generator.batch_size)

# Get the predicted class indices
predicted_class_indices = tf.argmax(predictions, axis=1).numpy()

# Get class labels
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())  # Reversing the dictionary
predicted_labels = [labels[k] for k in predicted_class_indices]

# Print predicted labels
for idx, label in enumerate(predicted_labels):
    print(f"Image {test_generator.filenames[idx]}: Predicted class -> {label}")
